## Импорт необходимых библиотек

С помощью библиотеки Dadata можно получить информацию о пункте по почтовому индексу

In [1]:
from dadata import Dadata
import numpy as np
import pandas as pd
import pickle
from tqdm.auto import tqdm

## Подготовка данных

Открываем файлы с данными и получаем список различных почтовых индексов

In [2]:
values = np.concatenate([
    pd.read_csv('./data/train_dataset_train.csv', index_col='id', low_memory=False).index_oper.values,
    pd.read_csv('./data/test_dataset_test.csv', index_col='id', low_memory=False).index_oper.values
])
values = np.unique(values[values != ' '].astype(float).astype(int))
values

array([     0, 101000, 102002, ..., 694920, 694923, 694928])

В день бесплатно можно выполнить органиченное число запросов. Используем лимит из первого аккаунта

In [3]:
token = "first_token"
dadata = Dadata(token)

In [4]:
data = []
for value in tqdm(values):
    data.append((
        value,
        dadata.suggest(name="postal_unit", query=str(value))
    ))

  0%|          | 0/21501 [00:00<?, ?it/s]

HTTPStatusError: Client error '403 Forbidden' for url 'https://suggestions.dadata.ru/suggestions/api/4_1/rs/suggest/postal_unit'
For more information check: https://httpstatuses.com/403

In [5]:
len(data)

14801

Информацию по оставшимся получим с помощью второго аккаунта

In [6]:
token = "second_token"
dadata = Dadata(token)

In [7]:
data2 = []
for value in tqdm(values[len(data):]):
    data2.append((
        value,
        dadata.suggest(name="postal_unit", query=str(value))
    ))

  0%|          | 0/6700 [00:00<?, ?it/s]

Сохраним данные, чтобы не потерять

In [8]:
assert len(data) + len(data2) == len(values)

In [9]:
all_data = data + data2

In [10]:
with open('./data/postal_data.pckl', 'wb') as f:
    pickle.dump(all_data, f)

## Создание датасета с дополнительной информацией

Открываем полученные данные

In [12]:
with open('./data/postal_data.pckl', 'rb') as f:
    postal_data = pickle.load(f)

Для некоторых индексов (и пропуска) не удалось получить информацию

In [13]:
bad_indexes = []

for i in range(len(postal_data)):
    if len(postal_data[i][1]) != 1:
        print(i, len(postal_data[i][1]))
        bad_indexes.append(postal_data[i][0])
    else:
        if str(postal_data[i][0]) != postal_data[i][1][0]['value']:
            print(i)
            bad_indexes.append(postal_data[i][0])
            
bad_indexes = set(bad_indexes)
bad_indexes

0 2
110 0
4068 0
4708 0
12788 0
16989 0
16990 0
16991 0
16992 0
16993 0


{0, 108957, 215057, 249287, 443322, 630301, 630302, 630303, 630311, 630321}

Пример собранных данных

In [14]:
postal_data[56]

(107065,
 [{'value': '107065',
   'unrestricted_value': 'г Москва, ул Уссурийская, д 11 к 1',
   'data': {'postal_code': '107065',
    'is_closed': False,
    'type_code': 'ГОПС',
    'address_str': 'г Москва, ул Уссурийская, д 11 к 1',
    'address_kladr_id': '7700000000000',
    'address_qc': '0',
    'geo_lat': 55.824784,
    'geo_lon': 37.820628,
    'schedule_mon': '08:00-21:00',
    'schedule_tue': '08:00-21:00',
    'schedule_wed': '08:00-21:00',
    'schedule_thu': '08:00-21:00',
    'schedule_fri': '08:00-21:00',
    'schedule_sat': '09:00-18:00',
    'schedule_sun': '09:00-14:00'}}])

Запишем долготу, широту, индекс региона

In [15]:
df = []
for v in tqdm(postal_data):
    index_oper = v[0]
    
    if index_oper in bad_indexes:
        df.append({
            'index_oper': index_oper,
        })
        continue

    v = v[1][0]['data']
    
    features = {
        'index_oper': index_oper,
        'geo_lat': v['geo_lat'],
        'geo_lon': v['geo_lon'],
    }
    
    if v['address_kladr_id'] is not None:
        features['region'] = v['address_kladr_id'][:2]
    
    df.append(features)

df = pd.DataFrame(df).set_index('index_oper')
df[['geo_lat', 'geo_lon']] = df[['geo_lat', 'geo_lon']].fillna(0)
df['region'] = df['region'].fillna('None')
df

  0%|          | 0/21501 [00:00<?, ?it/s]

,geo_lat,geo_lon,region
index_oper,,,
0,0.000000,0.000000,None
101000,55.763944,37.637281,77
102002,55.776148,37.662749,77
102007,55.776148,37.662749,77
102102,55.777570,37.660197,77
...,...,...,...
694910,49.158792,142.110316,65
694914,49.655256,142.182585,65
694920,49.079463,142.062490,65


Сохраним датасет

In [16]:
df.to_csv('./data/postal_data.csv')